## Table of Contents

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import time
import pandas as pd


sys.path.append('/home/class_NI2021/ctxctl_contrib_2023')

import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params
from bias_names import BIAS_NAMES

# Display plots inside Jupyter cell
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# Set the dots-per-inch (resolution) of the images
mpl.rcParams['figure.dpi'] = 90


In [2]:
device_name = 'dynpase_test' # pick any name
model,no_gui = ut.open_dynapse1(gui=False, sender_port=16258, receiver_port=12457, select_device=True)

ERROR: Address already in use, please re-run open_device()!
[0]:  Bus 1 Device 29 Dynapse1DevKit serial_number 00000027
[1]:  Bus 1 Device 27 Dynapse1DevKit serial_number 00000000
[2]:  Bus 1 Device 30 Dynapse1DevKit serial_number 00000020
[3]:  Bus 1 Device 26 Dynapse1DevKit serial_number 00000011
Sender port: tcp://0.0.0.0:16258
Receiver port: tcp://0.0.0.0:12457
Opened device name: Dynapse1DevKit
SamnaNode ID: 1
PythonNode ID: 2


In [5]:
def organize_results(events, df_spikes=None):
    # evts_n = np.array([[evt.timestamp, evt.neuron_id] for evt in events])
    timestamp_offset = 0 
    if df_spikes is None:
        df_spikes = pd.DataFrame()
    for evt in events:
        if timestamp_offset==0:
            timestamp_offset = evt.timestamp
        spikeData = dict(chip = evt.chip_id, 
                        core = evt.core_id,
                        neuron_id = evt.neuron_id,
                        times =(evt.timestamp-timestamp_offset)* 1e-6)
        df_spikes = pd.concat([df_spikes,pd.DataFrame(spikeData,index=[0])])
    return df_spikes

## Encoding signals with population codes

By now, you should be convinced that silicon neurons, like biological neurons, are heterogenous. However, averaging the activity of a population of heterogenous neurons, we can reduce the effect of variability.

In this tutorial, we go through different connectivity schemes: feed-forward, recurrent excitation and feedback inhibition. The aim is to highlight how and why we can combine all of them to create an EI network, a very common computational primitive in the brain.

This exercise will be mostly hands-free (minimal skeleton code) to get you used to coding from scratch (as will happen with the projects).


 ### Recurrent excitation
1. Allocate a cluster (i.e population) of 16 neurons in a single core.
2. Stimulate the cluster with a regular spike train of 100 Hz via excitatory synapse (you might want to use NMDA with threshold set to 0).
3. Set the network parameters to achive an average firing rate of 20 Hz.

4. Sweep over the input frequency and plot FF curve for the population activity.

In [3]:
emu_dur = 2 # in seconds
in_firing_rate = 100 # in hz

In [15]:
def generate_reg_sp_train(spikegen_gp, post_chip, rate, duration):
    # regular spike times
    spike_times = np.linspace(0, duration, duration * rate)
    # print(f"Spike times for a single spike generator ({len(spike_times)}):{spike_times}")

    # get global ids
    spgen_gids = ut.get_global_id_list(spikegen_gp.tuple_neuron_ids)

    # set spikegen ids global
    spike_ids = len(spike_times)* spgen_gids

    # set post chip
    # post_chip = DPI_neuron.chip_id
    target_chips = [post_chip]*len(spike_ids)

    return spike_times, spike_ids, target_chips
    

In [16]:

# set initial (proper) parameters
paramGroup = params.gen_clean_param_group()
for neu_chip_id in range(4):
    for neu_core_id in range(4):
        model.update_parameter_group(paramGroup, neu_chip_id, neu_core_id)

#-----------------------Network ----------------------------#
net_gen = n.NetworkGenerator()
# Allocate neurons of excitatory cluster
neu_chip_id = 0
neu_core_id = 1
n_exc = 16
neu_ids = np.arange(16, 16 + n_exc,1)

DPI_neuron =  n.NeuronGroup(neu_chip_id, neu_core_id, neu_ids)

# Allocate spikegen
spgen_chip_id = 0
spgen_core_id = 1
spgen_id = [0]
spikegen = n.NeuronGroup(spgen_chip_id ,spgen_core_id,spgen_id,True) 


# Connect all2all with AMPA synpase
syn = n.Synapses(spikegen, DPI_neuron, dyn1.Dynapse1SynType.NMDA, conn_type='all2all')
n.add_synapses(net_gen, syn)

print(f"Sanity check config:{net_gen.network}")

# apply configuration
current_config = net_gen.make_dynapse1_configuration()
model.apply_configuration(current_config)

#--------------------Set input spike train ----------------------#

# get FPGA spike_gen from model
fpga_spike_gen = model.get_fpga_spike_gen()
# # regular spike times
# spike_times = np.linspace(0, emu_dur, emu_dur *in_firing_rate)
# # print(f"Spike times for a single spike generator ({len(spike_times)}):{spike_times}")

# # get global ids
# spgen_gids = ut.get_global_id_list(spikegen.tuple_neuron_ids)

# # set spikegen ids global
# device_indices = len(spike_times)* spgen_gids

# # set post chip
# post_chip = DPI_neuron.chip_id
# target_chips = [post_chip]*len(device_indices)

isi_base = 900
repeat_mode = False
spike_times, dev_ids, target_chips = generate_reg_sp_train(spikegen, DPI_neuron.chip_id, in_firing_rate, emu_dur)
ut.set_fpga_spike_gen(fpga_spike_gen, spike_times, dev_ids, target_chips, isi_base, repeat_mode)

#-----------------Tune biases-------------------------------------#
paramGroup = dyn1.Dynapse1ParameterGroup()
    
#------------------------AMPA parameters----------------------#
# leakage of AMPA
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 7
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 255

# gain of AMPA
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 0
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 0

# weight of AMPA
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 0
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 0 

#------------------------NMDA parameters----------------------#
# first keep NMDA threshold to 0: i.e deactivate (AMPA_like) at coarse 0
paramGroup.param_map["IF_NMDA_N"].coarse_value = 0
paramGroup.param_map["IF_NMDA_N"].fine_value = 0

# Adjust weight of NMDA
paramGroup.param_map["PS_WEIGHT_EXC_S_N"].coarse_value = 6 
paramGroup.param_map["PS_WEIGHT_EXC_S_N"].fine_value = 100

# # NMDA time constant
paramGroup.param_map["NPDPIE_TAU_S_P"].coarse_value = 5
paramGroup.param_map["NPDPIE_TAU_S_P"].fine_value = 80

# NMDA gain
paramGroup.param_map["NPDPIE_THR_S_P"].coarse_value = 5
paramGroup.param_map["NPDPIE_THR_S_P"].fine_value = 40

#--------------------Neuron parameters--------------------------#
# Inject current to make neuron fire
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0

# Reduce neuron leakage
paramGroup.param_map["IF_TAU1_N"].coarse_value = 4
paramGroup.param_map["IF_TAU1_N"].fine_value = 80



model.update_parameter_group(paramGroup, DPI_neuron.chip_id, DPI_neuron.core_id)


#--------------------Monitor ----------------------#

graph, filter_node, sink_node = ut.create_neuron_select_graph(model, DPI_neuron.tuple_neuron_ids)



for i in range(2):
    # start gens
    fpga_spike_gen.start()

    # Start graph 
    graph.start()

    # Clear buffer
    sink_node.get_events()
    # get spikes every 2 seconds
    time.sleep(emu_dur + 0.5)
    events = sink_node.get_events()
    # process the events
    print(f"Events:{len(events)}    Mean rate:{len(events)/n_exc/(emu_dur+0.5)} Hz")
    graph.stop()
    fpga_spike_gen.stop()

    # Organize events
    results_df = organize_results(events)


Sanity check config:Post neuron (ChipId,coreId,neuronId): incoming connections [(preNeuron,synapseType), ...]
C0c1n29: [('C0c1s0', 'NMDA')]
C0c1n24: [('C0c1s0', 'NMDA')]
C0c1n18: [('C0c1s0', 'NMDA')]
C0c1n31: [('C0c1s0', 'NMDA')]
C0c1n26: [('C0c1s0', 'NMDA')]
C0c1n30: [('C0c1s0', 'NMDA')]
C0c1n27: [('C0c1s0', 'NMDA')]
C0c1n23: [('C0c1s0', 'NMDA')]
C0c1n21: [('C0c1s0', 'NMDA')]
C0c1n28: [('C0c1s0', 'NMDA')]
C0c1n16: [('C0c1s0', 'NMDA')]
C0c1n20: [('C0c1s0', 'NMDA')]
C0c1n25: [('C0c1s0', 'NMDA')]
C0c1n17: [('C0c1s0', 'NMDA')]
C0c1n19: [('C0c1s0', 'NMDA')]
C0c1n22: [('C0c1s0', 'NMDA')]

Events:753    Mean rate:18.825 Hz
Events:769    Mean rate:19.225 Hz


In [ ]:
# samna.device.close_device(model)